# 📜 **Finance Laws Retrieval-Augmented Generation (RAG) System**
### **Ironhack Data Science and Machine Learning Bootcamp**
📅 **Date:** February 5, 2025  
📅 **Submission Date:** February 7, 2025  
👩‍💻 **Authors:** Ginosca Alejandro Dávila & Natanael Santiago Morales  

---

## **📌 Introduction**
Retrieval-Augmented Generation (**RAG**) is an AI framework that enhances **information retrieval** by combining:
- A **vector database** to retrieve relevant documents.
- A **Large Language Model (LLM)** to generate contextualized responses.

This approach is widely used in **legal research, finance, customer support, and knowledge management**, improving accuracy and relevance in automated systems.

---

## **📌 Project Overview**
This project aims to build a **RAG system** focused on **finance laws and regulations**. By integrating **document retrieval** and **LLM-based generation**, the system will provide **accurate, contextualized answers** to legal and financial queries.

We will experiment with different **embedding models** (e.g., OpenAI’s `text-embedding-3-large` vs. `all-MiniLM-L6-v2`) and **vector databases** (e.g., ChromaDB) to optimize retrieval and response generation.

---

## **📂 Dataset Description**
- **Source:** A collection of **finance laws, regulations, and legal texts**.  
- **Format:** `.zip` file containing multiple `.pdf` files, each representing legal provisions, case law, or financial regulations.  

---

## **🎯 Goals**
✔ Develop a **RAG pipeline** specialized in finance law and legal text retrieval.  
✔ Compare **OpenAI embeddings** and **Sentence Transformers** for legal text retrieval.  
✔ Evaluate the system based on **retrieval accuracy** and **response relevance**.  
✔ *(Bonus)* Deploy the system as a **web application** using Streamlit or Flask.  
✔ *(Bonus)* Incorporate **legal case studies** and **financial reports** for richer context.  

---

🔹 **Let’s dive in and build our Finance RAG system! 🚀**


## **Data Selection**
We chose the **Financial Laws Collection**, a dataset containing **11 PDF documents** on **financial legislation in the European Union**.  

Initially, we considered the **Puerto Rico News Articles** dataset, but its **unstructured and unreadable format** made it unsuitable for this project.  

Each document in our chosen dataset varies in **length and content**, covering **legal provisions, case law, and financial regulations**.


## **Library Imports & Setup**

We begin by **importing and installing the necessary libraries** for the project. The core library used is **LangChain**, a framework designed to simplify the creation of applications that use **Large Language Models (LLMs)**.

### **Key Libraries Used**
- **`langchain_openai`** → Integrates OpenAI models via their SDK. Used for `OpenAIEmbeddings`.
- **`langchain_huggingface`** → Enables the use of **Hugging Face embeddings** within LangChain.
- **`langchain_chroma`** → Provides integration with **ChromaDB**, an AI-native vector database.
- **`langchain_community`** → Loads and processes PDF documents, splitting them into pages.
- **`langchain_text_splitters`** → Splits text into **manageable chunks** for retrieval-based processing.

Additionally, we use:
- **`enchant`** → A spell-checking library that helps correct and merge split words.
- **`openai`** → To connect with OpenAI's API for embeddings and response generation.

We also **load the OpenAI API key** and initialize the OpenAI client.


In [2]:
# Import necessary libraries
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

import os
import enchant
import re

from dotenv import load_dotenv
from IPython.display import Markdown


# Set your OpenAI API key
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")


from openai import OpenAI

client = OpenAI(
  api_key= API_KEY
)

## **Chunking the Text for Efficient Retrieval**

After extracting text from the PDF documents, we need to **split the text into chunks** before storing them in a vector database. This ensures **efficient retrieval** and **accurate responses**.

### **Text Splitter Selection**
We evaluated two splitting strategies:
1. **`CharacterTextSplitter`** → Splits text into **fixed-length chunks** with controlled overlap.  
2. **`RecursiveCharacterTextSplitter`** → Dynamically splits text but **often produces excessively small chunks**, reducing context.

### **Why We Chose `CharacterTextSplitter`**
✔ **Preserves context** using an **overlap of 200 characters**.  
✔ **Prevents excessive fragmentation**, ensuring chunks retain **complete, meaningful information**.  
✔ **Offers predictable behavior**, making retrieval tuning easier.

### **Implementation Steps**
1. **Initialized an empty list (`all_docs[]`)** to store processed chunks.
2. **Created a text splitter** (`CharacterTextSplitter`) with `chunk_size=1000` and `chunk_overlap=200`.
3. **Looped through the PDF files**, loading each document and splitting them into **pages**.
4. **Further split each page into smaller chunks**, maintaining meaningful context.
5. **Stored the processed chunks in `all_docs[]`**, preparing them for embedding and retrieval.

The following code block implements this process.


In [ ]:
# Directory containing the PDF files
document_dir = "./data/Finance_laws/EurLex_v1"

# Initialize an empty list to hold all documents
all_docs = []

# Create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Loop through all PDF files in the directory
for filename in os.listdir(document_dir):
    # Checking that we only take pdf files
    if filename.endswith(".pdf"):
        # after getting the file name, we add it to the path
        file_path = os.path.join(document_dir, filename)
        # Load the document and split it into pages
        pages = PyPDFLoader(file_path).load_and_split()
        # Split the text content of each page into smaller chunks
        docs = text_splitter.split_documents(pages)
        # Add the smaller chunks to the all_docs list
        all_docs.extend(docs)

print(f"Number of documents: {len(all_docs)}")

Number of documents: 1561


## **Examining a Sample Chunk**

After loading and chunking all the documents, we now examine a **sample chunk** from `all_docs[]`.  

Each chunk is represented as a **LangChain `Document` object**, which contains:
- **Metadata** → Includes the **document source** (file path) and **page number**.
- **Page Content** → The actual **text extracted** from that chunk.

Below is an example of how a chunk is structured:

```python
[
    Document(
        metadata={
            'source': './data/Finance_laws/EurLex_v1/AMLD_EURLEX.pdf', # Path to the document
            'page': 0,                                                  # Page number (from the document)
            'page_label': '1'                                           # Page number assigned by the text splitter.
        },
        page_content=                                                   # Text from the chunk
            "Chunk contents"
    )
]


In [ ]:
all_docs[0]

Document(metadata={'source': './data/Finance_laws/EurLex_v1\\AMLD_EURLEX.pdf', 'page': 0, 'page_label': '1'}, page_content="DIRECTIVES \nDIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL \nof 20 May 2015 \non the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or \nter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the \nCouncil, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and \nCommission Directiv e 2006/70/EC \n(T ext with EEA relevance) \nTHE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, \nHa ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, \nHa ving regar d to the proposal from the European Commission, \nAf ter transmission of the draf t legislative act to the national parliaments, \nHa ving regar d to the opinion of the European Central Bank (\n1\n), \nHa ving 

## **Evaluating the Quality of Chunks**

To assess our chunking strategy, we print **the first chunk** of `all_docs[]`.

### **Key Checks**
✔ **Does each chunk retain full sentences?**  
✔ **Are words improperly split or cut off?**  
✔ **Are there excessive line breaks or formatting errors?**  

Identifying **text anomalies** early ensures clean data for embedding and retrieval.


In [ ]:
print(all_docs[0].page_content)
display(Markdown(all_docs[0].page_content))

DIRECTIVES 
DIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL 
of 20 May 2015 
on the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or 
ter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the 
Council, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and 
Commission Directiv e 2006/70/EC 
(T ext with EEA relevance) 
THE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, 
Ha ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, 
Ha ving regar d to the proposal from the European Commission, 
Af ter transmission of the draf t legislative act to the national parliaments, 
Ha ving regar d to the opinion of the European Central Bank (
1
), 
Ha ving regar d to the opinion of the European Economic and Social Committ ee (
2
), 
A cting in accordance with the ordinar y legislative procedur

DIRECTIVES 
DIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL 
of 20 May 2015 
on the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or 
ter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the 
Council, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and 
Commission Directiv e 2006/70/EC 
(T ext with EEA relevance) 
THE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, 
Ha ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, 
Ha ving regar d to the proposal from the European Commission, 
Af ter transmission of the draf t legislative act to the national parliaments, 
Ha ving regar d to the opinion of the European Central Bank (
1
), 
Ha ving regar d to the opinion of the European Economic and Social Committ ee (
2
), 
A cting in accordance with the ordinar y legislative procedure (
3
), 
Whereas: 
(1)  Flo ws of illicit mone y can damage the integrit y , stability and reputation of the financial sector , and threaten the 
internal market of the U nion as well as intern ational development. Mone y launder ing, terror ism financing and 
org anised cr ime remain significant problems which should be addressed at U nion level. In addition to fur ther 
developing the cr iminal la w approach at Union level, target ed and propor tionat e prevention of the use of the 
financia l system f or the pur poses of money launder ing and terr or ist financing is indispensable and can produce 
compl ementar y results. 
(2)  The soundness, integr ity and stability of credit institutions and fi nancial institutions, and confidence in the 
financia l system as a whole could be ser iously jeopardised by the eff or ts of cr iminals and their associates to 
disguise the or igin of cr iminal proceeds or to c hannel lawful or illicit money f or terr or ist pur poses. In order to 
f acilitate their cr iminal activities, mone y launderers and financer s of te r ror ism could tr y to take advantage of the 
freedom of capital mo vements and the freedom to supply fi nancial ser vices whic h the U nion's integrat ed fi nancial 
area entails. Theref ore, cer tain coordinating measures are necessar y at U nion level. A t the same time, the 
objectives of prot ecting society from cr ime and protectin g the stability and integrit y of the Union's financia l 
system should be balanced ag ainst the need to create a regulatory en vironment that allo ws compan ies to gro w 
their businesses without incur r ing dispropor tionate compl iance costs. 
(3)  This Directive is the f our th directive to address the threat of mone y launder ing. Council Directive 91/308/EEC (
4
) 
defined mone y launder ing in term s of dr ugs off ences and imposed obligations solely on the financial sector . 
5.6.2015 L 141/73 Official Jour nal of the European Union EN     
(
1
) OJ C 166, 12.6.2013, p. 2. 
(
2
) OJ C 271, 19.9.2013, p. 31. 
(
3
) P osition of the European P arliament of 11 March 2014 (not yet published in the Offi cial Jour nal) and position of the Council at first 
reading of 20 Apr il 2015 (not yet published in the Offi cial Jour nal). P osition of the European P arliament of 20 Ma y 2015 (not yet 
published in the Official Jour nal). 
(
4
) Council Directive 91/308/EEC of 10 June 1991 on prevention of the use of the f inancial syst em f or the pur pose of money launder ing 
(OJ L 166, 28.6.1991, p. 77).

## **Text Cleaning & Preprocessing**

To ensure high-quality retrieval, we implemented a **text cleaning function** that:
- **Removes unnecessary line breaks**, replacing them with spaces.
- **Merges split words** that were incorrectly separated due to PDF formatting.
- **Ensures proper spacing and punctuation formatting** for readability.

### **How the Cleaning Function Works**
1. **Tokenizes text into individual words and symbols**.  
2. **Attempts to merge adjacent words** that were mistakenly split.  
3. **Verifies word validity** using the **`enchant`** dictionary.  
   - If the merged word is valid, it replaces the split parts.  
   - If the merged word is invalid, the function keeps the original split.  
4. **Maintains punctuation and structural integrity** to ensure clean, readable text.  

This process significantly **improves text consistency**, making it more suitable for retrieval and embedding.


In [ ]:
import enchant
import re

# Load the English dictionary
d = enchant.Dict("en_US")

# Function to correct split words
def correct_split_words(sentence):
    # Replace newlines with spaces
    sentence = sentence.replace("\n", " ")

    words = sentence.split()  # Tokenize the sentence
    i = 0
    corrected_sentence = []

    while i < len(words) - 1:
        merged_word = words[i] + words[i+1]  # Merge adjacent words
        stripped_merged_word = re.sub(r'[^a-zA-Z]', '', merged_word) #remove symbols from merged word using regex

        # check if stripped merged word is empty
        if stripped_merged_word == '':
            corrected_sentence.append(words[i])
            i += 1
        elif d.check(stripped_merged_word):  # Check if merged word is in the dictionary
            corrected_sentence.append(merged_word)  # Add corrected word
            i += 2  # Skip the next word since it's already merged
        else:
            corrected_sentence.append(words[i])  # Add unmerged word
            i += 1  # Move to the next word

    # Add the last word if it wasn't merged
    if i < len(words):
        corrected_sentence.append(words[i])

    corrected_sentence = " ".join(corrected_sentence)

    corrected_sentence = re.sub(r'([,.")(])(?=\D)', r'\1 ', corrected_sentence)
    corrected_sentence = re.sub(r'\s+([,.)])', r'\1', corrected_sentence)
    corrected_sentence = re.sub(r'\(\s+', '(', corrected_sentence)
    corrected_sentence = re.sub(r'\s+', ' ', corrected_sentence).strip()
    corrected_sentence = re.sub(r'(\d)([A-Za-z])', r'\1 \2', corrected_sentence)
    corrected_sentence = re.sub(r'([A-Za-z]{2,})(\d+)', r'\1 \2', corrected_sentence)
    corrected_sentence = re.sub(r'(\w)(\()(?!s\))', r'\1 \2', corrected_sentence)

    return corrected_sentence

## **Applying Text Cleaning to All Chunks**
Now that we have defined the **`correct_split_words()` function**, we apply it to **every chunk in `all_docs[]`**.

This ensures that:

✔ **All retrieved text is well-formatted** before embedding.  
✔ **Errors from extraction are minimized.**  
✔ **Retrieval quality is improved** since the model will process clearer text.


In [ ]:
# Apply the function to all documents in `all_docs[]`
for doc in all_docs:
    doc.page_content = correct_split_words(doc.page_content)

print(all_docs[0].page_content)

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can d

## **Verifying Cleaned Text**
We now print the **same chunk as before** after applying text cleaning.

### **Improvements Observed**
✔ **No more unwanted line breaks**.  
✔ **Words that were incorrectly split are now properly merged**.  
✔ **The text is more structured and readable**.  

This cleaned version will be **embedded into our vector database** for retrieval.


In [ ]:
print(all_docs[0].page_content)
display(Markdown(all_docs[0].page_content))

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can d

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can damage the integrity, stability and reputation of the financial sector, and threaten the internal market of the Union as well as international development. Money laundering, terrorism financing and org anised crime remain significant problems which should be addressed at Union level. In addition to further developing the criminal law approach at Union level, targeted and propor tionat e prevention of the use of the financial system for the purposes of money laundering and terror ist financing is indispensable and can produce compl ementar y results. (2) The soundness, integrity and stability of credit institutions and financial institutions, and confidence in the financial system as a whole could be seriously jeopardised by the eff orts of criminals and their associates to disguise the origin of criminal proceeds or to channel lawful or illicit money for terror ist purposes. In order to facilitate their criminal activities, money launderers and financer s of ter ror ism could try to take advantage of the freedom of capital movements and the freedom to supply financial services which the Union's integrated financial area entails. Therefore, certain coordinating measures are necessary at Union level. At the same time, the objectives of protecting society from crime and protecting the stability and integrity of the Union's financial system should be balanced against the need to create a regulatory environment that allows companies to grow their businesses without incur ring disproportionate compliance costs. (3) This Directive is the four th directive to address the threat of money laundering. Council Directive 91/308/EEC (4) defined money laundering in terms of drugs off ences and imposed obligations solely on the financial sector. 5.6.2015 L 141/73 Official Journal of the European Union EN (1) OJ C166, 12.6.2013, p. 2. (2) OJ C271, 19.9.2013, p. 31. (3) P osition of the European Parliament of 11 March 2014 (not yet published in the Official Journal) and position of the Council at first reading of 20 April 2015 (not yet published in the Official Journal). Position of the European Parliament of 20 May 2015 (not yet published in the Official Journal). (4) Council Directive 91/308/EEC of 10 June 1991 on prevention of the use of the financial system for the purpose of money laundering (OJ L166, 28.6.1991, p. 77).

## **Embedding the Text for Retrieval**
We generate **vector embeddings** for each document chunk using **two different models**:

1. **OpenAI Embeddings (`text-embedding-3-large`)**  
   - Commercial model known for **high accuracy and performance**.  
2. **Hugging Face Embeddings (`all-MiniLM-L6-v2`)**  
   - Open-source alternative that balances **efficiency and quality**.

Using **two different embedding methods** allows us to compare performance and optimize retrieval accuracy.


In [4]:
# Create the embeddings function

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=API_KEY)
print("Initialized OpenAI embeddings.")

sentence_transformer_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Initialized SentenceTransformer embeddings.")

Initialized OpenAI embeddings.

Initialized SentenceTransformer embeddings.


## **Storing the Chunks in the Vector Database**

After initializing the embeddings, we created a vector database using **ChromaDB** and stored the embeddings of each chunk in the database. 

We created **two databases**: 
1. One for the **OpenAI** embeddings and 
2. One for the **Hugging Face** embeddings.

In [ ]:
# Load all documents into Chroma
db = Chroma.from_documents(all_docs, sentence_transformer_embeddings, persist_directory="./chroma_db_clean_huggingface")
print('All documents loaded and embedded.(huggingface)')

db_openai = Chroma.from_documents(all_docs, openai_embeddings, persist_directory="./chroma_db_clean_openai")
print('All documents loaded and embedded.(openai)')

All documents loaded and embedded.(huggingface)
All documents loaded and embedded.(openai)


## **Retrieving Similar Chunks**
We use **vector similarity search** to find the most relevant document chunks for a given query.

### **Retrieval Process**
1. **Convert the user query into an embedding**.
2. **Compare it against stored document embeddings**.
3. **Retrieve the top `k=5` most similar chunks**.

Below, we retrieve **3 most relevant chunks** using:
- **Hugging Face embeddings** (`all-MiniLM-L6-v2`).


In [ ]:
# Example query
user_question = "What are the key points of financial legislation in Europe?"
docs = db.similarity_search(user_question, k=5)

# Print results
for doc in docs[0:3]:
    # print(doc.page_content, '\n')
    display(Markdown(doc.page_content))
    print("*"*80, '\n')

I (Legislative acts) REGULATIONS REGULATION (EU) No 575/2013 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Having regard to the opinion of the European Economic and Social Committee (2), Acting in accordance with the ordinary legislative procedure, Whereas:(1) The G-20 Declaration of 2 April 2009 on Strengthening of the Financial System called for internationally consistent efforts that are aimed at strengthening trans­ parency, accountability and regulation by improving the quantity and quality of capital in the banking system once the economic recovery is assured. That declaration also called for introduction of a supplementary non-risk based measure to contain the build-up of leverage in the banking system, and the development of a framework for stronger liquidity buffers. In response to the mandate given by the G-20, in September 2009 the Group of Central Bank Governors and Heads of Supervision (GHOS), agreed on a number of measures to strengthen the regulation of the banking sector. Those measures were endorsed by the G-20 leaders at their Pittsburgh Summit of 24-25 September 2009 and were set out in detail in December 2009. In July and September 2010, GHOS issued two further announcements on design and calibration of those new measures, and in December 2010, the Basel Committee on Banking Supervision (BCBS) published the final measures, that are referred to as the Basel III framework. (2) The High Level Group on Financial Supervision in the EU chaired by Jacques de Larosière (the " de Larosière group") invited the Union to develop a more harmonised set of financial regulations. In the context of the future European supervisory architecture, the European Council of 18 and 19 June 2009 also stressed the need to establish a' European Single Rule Book' applicable to all credit institutions and investment firms in the internal market. (3) As stated in the de Larosière group's report of 25 February 2009 (the" de Larosière report"), " a Member State should be able to adopt more stringent national regulatory measures considered to be domes­ tically appropriate for safeguarding financial stability as long as the principles of the internal market and agreed minimum core standards are respected". (4) Directive 2006/48/EC of the European Parliament and of the Council of 14 June 2006 relating to the taking up and pursuit of the business of credit institutions (3) and Directive 2006/49/EC of the European Parliament and of EN 27.6.2013 Official Journal of the European Union L176/1 (1) OJ C105, 11.4.2012, p. 1. (2) OJ C68, 6.3.2012, p. 39. (3) OJ L177, 30.6.2006, p. 1.

******************************************************************************** 



(121) In order to take account of market developments and experience in the application of this Regulation, the Commission should be required to submit reports to the European Parliament and to the Council, together with legislative proposals, where appropriate, on the possible effect of capital requirements on the economic cycle of minimum, own funds requirements for exposures in the form of covered bonds, large exposures, liquidity requirements, leverage, exposures to transferred credit risk, counterparty credit risk and the original exposure method, retail exposures, on the definition of eligible capital, and the level of application of this Regu ­ lation. (122) The primary purpose of the legal framework for credit institutions should be to ensure the operation of vital services to the real economy while limiting the risk of moral hazard. The structural separation of retail and investment banking activities within a banking group could be one of the key tools to support this objective. No provision in the current regulatory framework should therefore prevent the introduction of measures to effect such a separation. The Commission should be required to analyse the issue of structural separation in the Union and submit a report, together with legislative proposals, if appropriate, to the European Parliament and the Council. (123) Similarly, with a view to protecting depositors and preserving financial stability, Member States should also be permitted to adopt structural measures that require credit institutions authorised in that Member State to reduce their exposures to different legal entities depending on their activities, irrespective of where those activities are located. However, because such measures could have a negative impact by fragmenting the internal market, they should only be approved subject to strict conditions pending the entry into force of a future legal act explicitly harmonising such measures. (124) In order to specify the requirements set out in this Regu ­ lation, the power to adopt acts in accordance with Article 290 TFEU should be delegated to the Commission in respect of technical adjustments to this Regulation to clarify definitions to ensure uniform appli ­cation of this Regulation or to take account of devel ­ opments on financial markets, to align terminology on, and frame definitions in accordance with, subsequent relevant acts, to adjust the provisions of this Regulation on own funds to reflect developments in accounting standards or Union law, or with regard to the convergence of supervisory practices, to expand the lists of exposure classes for the purposes of the Stan­ dardised Approach or the IRB Approach to take account of developments on financial markets, to adjust certain amounts relevant to those exposure classes to take into account the effects of inflation; to adjust the list and classification of off- balance sheet items and to adjust specific provisions and technical criteria on the treatment of counterparty credit risk, the Standardised Approach and the IRB Approach, credit risk mitigation, securitisation, operational risk, market risk, liquidity, leverage and disclosure in order to take account of developments on financial markets or in accounting standards or Union law, or with regard to the convergence of supervisory practices and risk measurement and to take account of the outcome of the review of various matters relating to the scope of Directive 2004/39/EC. (125) The power to adopt acts in accordance with Article 290 TFEU should also be delegated to the Commission in respect of prescribing a temporary reduction in the level of own funds or risk weights specified under this Regulation in order to take account of specific circum ­stances, to clarify the exemption of certain exposures from the application of provisions of this Regulation on large exposures, to specify amounts relevant to the

******************************************************************************** 



(4) Directive 2004/39/EC of the European Parliament and of the Council of 21 April 2004 on markets in financial instruments (1) allows investment firms authorised by the competent authorities of their home Member State and supervised by the same authorities to establish branches and provide services freely in other Member States. That Directive accordingly provides for the coordination of the rules governing the authorisation and pursuit of the business of investment firms. It does not, however, establish the amounts of the initial capital of such firms ora common framework for monitoring the risks incurred by them, which should be provided by this Directive. (5) This Directive should constitute the essential instrument for the achievement of the internal market from the point of view of both the freedom of establishment and the freedom to provide financial services in the field of credit institutions. (6) The smooth operation of the internal market requires not only legal rules but also close and regular cooperation and significantly enhanced convergence of regulatory and supervisory practices between the competent authorities of the Member States. (7) Regulation (EU) No 1093/2010 of the European Parliament and of the Council (2) established the European Supervisory Authority (European Banking Authority) (" EBA"). This Directive should take into account the role and function of EBA set out in that Regulation and the procedures to be followed when conferring tasks on EBA. (8) Given the increase of tasks conferred on EBA by this Directive and by Regulation (EU) No 575/2013, the European Parliament, the Council and the Commission should ensure that adequate human and financial resources are made available. (9) Asa first step towards a banking union, a single super­ visory mechanism (SSM) should ensure that the Union's policy relating to the prudential supervision of credit institutions is implemented in a coherent and effective manner, that the single rulebook for financial services is applied in the same manner to credit institutions in all Member States concerned, and that those credit insti ­ tutions are subject to supervision of the highest quality, unfettered by other, non-prudential considerations. An SSM is the basis for the next steps towards a banking union. This reflects the principle that any introduction of common intervention mechanisms in the event of a crisis should be preceded by common controls to reduce the likelihood that such intervention mechanisms will have to be used. The European Council noted in its conclusions of 14 December 2012 that " the Commission will submit in the course of 2013 a proposal fora single resolution mechanism for Member States participating in the SSM, to be examined by co-legislators as a matter of priority with the intention of adopting it during the current parliamentary cycle. ". The integration of the financial framework could be further enhanced through the setting up of a single resolution mechanism, including appropriate and effective backstop arrangements to ensure that bank resolution decisions are taken swiftly, impartially and in the best interests of all concerned. (10) The conferral of supervisory tasks on the European Central Bank (ECB) for some of the Member States should be consistent with the framework of the European System of Financial Supervision setup in 2010 and its underlying objective to develop the single rulebook and enhance convergence of supervisory practices across the Union as a whole. The ECB should carryout its tasks subject to and in compliance with any relevant primary and secondary Union law, Commission decisions in the areas of State aid, competition rules and merger control and the single rulebook applying to all Member States. EBA is entrusted with developing draft technical standards and guidelines and recommendations

******************************************************************************** 



## **Retrieving Similar Chunks**
We use **vector similarity search** to find document chunks most relevant to a given query.

### **Retrieval Process**
1. **Convert the user query into an embedding**.
2. **Compare it against stored document embeddings**.
3. **Retrieve the top `k=5` most similar chunks**.

Below, we retrieve **3 most relevant chunks** using:
- **OpenAI embeddings** (`text-embedding-3-large`).


In [ ]:
# Example query
user_question = "What are the key points of financial legislation in Europe?"
docs_openai = db.similarity_search(user_question, k=5)

# Print results
for doc in docs_openai[0:3]:
    # print(doc.page_content, '\n')
    display(Markdown(doc.page_content))
    print("*"*80, '\n')

I (Legislative acts) REGULATIONS REGULATION (EU) No 575/2013 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Having regard to the opinion of the European Economic and Social Committee (2), Acting in accordance with the ordinary legislative procedure, Whereas:(1) The G-20 Declaration of 2 April 2009 on Strengthening of the Financial System called for internationally consistent efforts that are aimed at strengthening trans­ parency, accountability and regulation by improving the quantity and quality of capital in the banking system once the economic recovery is assured. That declaration also called for introduction of a supplementary non-risk based measure to contain the build-up of leverage in the banking system, and the development of a framework for stronger liquidity buffers. In response to the mandate given by the G-20, in September 2009 the Group of Central Bank Governors and Heads of Supervision (GHOS), agreed on a number of measures to strengthen the regulation of the banking sector. Those measures were endorsed by the G-20 leaders at their Pittsburgh Summit of 24-25 September 2009 and were set out in detail in December 2009. In July and September 2010, GHOS issued two further announcements on design and calibration of those new measures, and in December 2010, the Basel Committee on Banking Supervision (BCBS) published the final measures, that are referred to as the Basel III framework. (2) The High Level Group on Financial Supervision in the EU chaired by Jacques de Larosière (the " de Larosière group") invited the Union to develop a more harmonised set of financial regulations. In the context of the future European supervisory architecture, the European Council of 18 and 19 June 2009 also stressed the need to establish a' European Single Rule Book' applicable to all credit institutions and investment firms in the internal market. (3) As stated in the de Larosière group's report of 25 February 2009 (the" de Larosière report"), " a Member State should be able to adopt more stringent national regulatory measures considered to be domes­ tically appropriate for safeguarding financial stability as long as the principles of the internal market and agreed minimum core standards are respected". (4) Directive 2006/48/EC of the European Parliament and of the Council of 14 June 2006 relating to the taking up and pursuit of the business of credit institutions (3) and Directive 2006/49/EC of the European Parliament and of EN 27.6.2013 Official Journal of the European Union L176/1 (1) OJ C105, 11.4.2012, p. 1. (2) OJ C68, 6.3.2012, p. 39. (3) OJ L177, 30.6.2006, p. 1.

******************************************************************************** 



(121) In order to take account of market developments and experience in the application of this Regulation, the Commission should be required to submit reports to the European Parliament and to the Council, together with legislative proposals, where appropriate, on the possible effect of capital requirements on the economic cycle of minimum, own funds requirements for exposures in the form of covered bonds, large exposures, liquidity requirements, leverage, exposures to transferred credit risk, counterparty credit risk and the original exposure method, retail exposures, on the definition of eligible capital, and the level of application of this Regu ­ lation. (122) The primary purpose of the legal framework for credit institutions should be to ensure the operation of vital services to the real economy while limiting the risk of moral hazard. The structural separation of retail and investment banking activities within a banking group could be one of the key tools to support this objective. No provision in the current regulatory framework should therefore prevent the introduction of measures to effect such a separation. The Commission should be required to analyse the issue of structural separation in the Union and submit a report, together with legislative proposals, if appropriate, to the European Parliament and the Council. (123) Similarly, with a view to protecting depositors and preserving financial stability, Member States should also be permitted to adopt structural measures that require credit institutions authorised in that Member State to reduce their exposures to different legal entities depending on their activities, irrespective of where those activities are located. However, because such measures could have a negative impact by fragmenting the internal market, they should only be approved subject to strict conditions pending the entry into force of a future legal act explicitly harmonising such measures. (124) In order to specify the requirements set out in this Regu ­ lation, the power to adopt acts in accordance with Article 290 TFEU should be delegated to the Commission in respect of technical adjustments to this Regulation to clarify definitions to ensure uniform appli ­cation of this Regulation or to take account of devel ­ opments on financial markets, to align terminology on, and frame definitions in accordance with, subsequent relevant acts, to adjust the provisions of this Regulation on own funds to reflect developments in accounting standards or Union law, or with regard to the convergence of supervisory practices, to expand the lists of exposure classes for the purposes of the Stan­ dardised Approach or the IRB Approach to take account of developments on financial markets, to adjust certain amounts relevant to those exposure classes to take into account the effects of inflation; to adjust the list and classification of off- balance sheet items and to adjust specific provisions and technical criteria on the treatment of counterparty credit risk, the Standardised Approach and the IRB Approach, credit risk mitigation, securitisation, operational risk, market risk, liquidity, leverage and disclosure in order to take account of developments on financial markets or in accounting standards or Union law, or with regard to the convergence of supervisory practices and risk measurement and to take account of the outcome of the review of various matters relating to the scope of Directive 2004/39/EC. (125) The power to adopt acts in accordance with Article 290 TFEU should also be delegated to the Commission in respect of prescribing a temporary reduction in the level of own funds or risk weights specified under this Regulation in order to take account of specific circum ­stances, to clarify the exemption of certain exposures from the application of provisions of this Regulation on large exposures, to specify amounts relevant to the

******************************************************************************** 



(4) Directive 2004/39/EC of the European Parliament and of the Council of 21 April 2004 on markets in financial instruments (1) allows investment firms authorised by the competent authorities of their home Member State and supervised by the same authorities to establish branches and provide services freely in other Member States. That Directive accordingly provides for the coordination of the rules governing the authorisation and pursuit of the business of investment firms. It does not, however, establish the amounts of the initial capital of such firms ora common framework for monitoring the risks incurred by them, which should be provided by this Directive. (5) This Directive should constitute the essential instrument for the achievement of the internal market from the point of view of both the freedom of establishment and the freedom to provide financial services in the field of credit institutions. (6) The smooth operation of the internal market requires not only legal rules but also close and regular cooperation and significantly enhanced convergence of regulatory and supervisory practices between the competent authorities of the Member States. (7) Regulation (EU) No 1093/2010 of the European Parliament and of the Council (2) established the European Supervisory Authority (European Banking Authority) (" EBA"). This Directive should take into account the role and function of EBA set out in that Regulation and the procedures to be followed when conferring tasks on EBA. (8) Given the increase of tasks conferred on EBA by this Directive and by Regulation (EU) No 575/2013, the European Parliament, the Council and the Commission should ensure that adequate human and financial resources are made available. (9) Asa first step towards a banking union, a single super­ visory mechanism (SSM) should ensure that the Union's policy relating to the prudential supervision of credit institutions is implemented in a coherent and effective manner, that the single rulebook for financial services is applied in the same manner to credit institutions in all Member States concerned, and that those credit insti ­ tutions are subject to supervision of the highest quality, unfettered by other, non-prudential considerations. An SSM is the basis for the next steps towards a banking union. This reflects the principle that any introduction of common intervention mechanisms in the event of a crisis should be preceded by common controls to reduce the likelihood that such intervention mechanisms will have to be used. The European Council noted in its conclusions of 14 December 2012 that " the Commission will submit in the course of 2013 a proposal fora single resolution mechanism for Member States participating in the SSM, to be examined by co-legislators as a matter of priority with the intention of adopting it during the current parliamentary cycle. ". The integration of the financial framework could be further enhanced through the setting up of a single resolution mechanism, including appropriate and effective backstop arrangements to ensure that bank resolution decisions are taken swiftly, impartially and in the best interests of all concerned. (10) The conferral of supervisory tasks on the European Central Bank (ECB) for some of the Member States should be consistent with the framework of the European System of Financial Supervision setup in 2010 and its underlying objective to develop the single rulebook and enhance convergence of supervisory practices across the Union as a whole. The ECB should carryout its tasks subject to and in compliance with any relevant primary and secondary Union law, Commission decisions in the areas of State aid, competition rules and merger control and the single rulebook applying to all Member States. EBA is entrusted with developing draft technical standards and guidelines and recommendations

******************************************************************************** 



## **Loading the Databases**

To efficiently retrieve relevant legal documents, we load two separate databases, each utilizing a different embedding model. This allows for flexibility in selecting the appropriate model for different use cases.

### **Database Loading Process**
1. **Hugging Face Embeddings Database**: 
   - The first database leverages **Hugging Face embeddings** for similarity search.
   - We load it from the specified persistence directory: `./chroma_db_clean_huggingface`.

2. **OpenAI Embeddings Database**: 
   - The second database uses **OpenAI embeddings** for similarity search.
   - It is loaded from the directory: `./chroma_db_clean_openai`.


In [5]:
print("Loading the databases")

persist_directory_huggingface = "./chroma_db_clean_huggingface"
db = Chroma(persist_directory=persist_directory_huggingface, embedding_function=sentence_transformer_embeddings)
print("Huggingface database loaded.")

persist_directory_openai = "./chroma_db_clean_openai"
db_openai = Chroma(persist_directory=persist_directory_openai, embedding_function=openai_embeddings)
print("OpenAI database loaded.")

Loading the databases
Huggingface database loaded.
OpenAI database loaded.


## **Formatting Retrieved Chunks for Prompting**
To **generate responses**, we need to format the retrieved chunks into a structured prompt.

The `_get_document_prompt()` function:
1. Extracts **metadata** (source file, page number).
2. Combines **relevant document chunks** into a single text block.
3. Structures the information **so that the LLM can generate more accurate answers**.

This ensures that responses are **contextually accurate and properly cited**.


In [6]:
# Function to combine documents into a single string
def _get_document_prompt(docs):
    """
    Combine the content of multiple documents into a single string

    Args:
    docs (list): A list of Document objects

    Returns:
    str: A string containing the content of all documents
    """

    prompt = ""
    for doc in docs:
        # Extract metadata for citation
        source = doc.metadata.get('source', 'Unknown source')
        page = doc.metadata.get('page', 'Unknown page')

        # Include both metadata and content in the prompt
        if page != 'Unknown page':
            prompt += f"Source: {source}, Page: {page+1}\n{doc.page_content}\n\n"
        else:
            prompt += f"Source: {source}, Page: {page}\n{doc.page_content}\n\n"

    return prompt


To **provide context for answering user questions**, the `get_context()` function retrieves relevant document chunks based on the query:

1. **Arguments**:
    - `user_question`: The **query** posed by the user.
    - `k`: The number of **top documents** to retrieve.
    - `openai`: A **flag** indicating whether to use **OpenAI** embeddings or **SentenceTransformer** embeddings.

2. **Logic**:
    - Depending on the **embedding model**, the function retrieves the most **similar documents** using Chroma.
    - The function then **formats the retrieved content** using `_get_document_prompt()` to provide **context for the response**.

This ensures that the generated answer is based on the most relevant chunks of information.


In [8]:
def get_context (user_question, k, openai):
    """
    Get the context for the user question

    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings

    Returns:
    str: The context for the user question
    """
    
    if openai:
        docs = db_openai.similarity_search(user_question, k)
    else:
        docs = db.similarity_search(user_question, k)
    return _get_document_prompt(docs)

The `get_prompt()` function is designed to create a **structured prompt** for answering legal questions based on **European Union Financial Laws**.

1. **Arguments**:
    - `user_question`: The **user’s query**.
    - `k`: The number of **documents to retrieve** (default: 10).
    - `openai`: A flag for using either **OpenAI** embeddings or **SentenceTransformer** embeddings.

2. **Logic**:
    - The function generates a **prompt** with the following sections:
      - **Introduction**: Clarifies the chatbot’s role as a legal assistant.
      - **Context**: Provides relevant legal documentation, retrieved using `get_context()`.
      - **Restrictions**: Outlines the chatbot’s behavioral **constraints** (e.g., only factual responses).
      - **Task**: Specifies the **answering structure** and the **format** for responding.

3. **Output**:
    - A **well-formatted prompt** containing the legal context, guidelines for responding, and a structure for the response, including **citation** of the source documents and pages.

This ensures that the chatbot gives precise, legally accurate, and properly cited answers.


In [9]:
def get_prompt (user_question, k=10, openai=False):
    """
    Generate a prompt for the user question

    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings

    Returns:
    str: The prompt for the user question
    """

    return f"""
## INTRODUCTION
You are a Chatbot designed to help lawyers with European Union Financial Laws. 
The user asked: "{user_question}"

## CONTEXT
Legal Documentation related to the users' question is provided below:
'''
{get_context(user_question, k, openai)}
'''

## RESTRICTIONS
Refer to the laws by their name, number, or article. 
Be clear, transparent, and factual: only state what is in the context without providing opinions or subjectivity.
Answer the question based solely on the context above; if you do not know the answer, be clear with the user that you do not know.
Only respond to questions related to the laws, avoiding jokes, offensive remarks, and discussions on religion or sexuality, unless discussed in the context.
If the user does not provide sufficient context, do not answer and instead ask for more information on what the user wants to know.

## TASK
First, answer directly to the user, if possible.
Second, reference the source of the information you provided in parentheses, next to the answer.
Lastly, answer in Markdown format.

## RESPONSE STRUCTURE:
'''
# [Answer Title]
[answer text (reference). Keep answering and (reference) until you have answered all parts of the question.]

Source:
- From `[Document name], page [page number]` of the European Union Finance Laws, from the Official Journal of the European Union
- From `[Document name], page [page number]` of the European Union Finance Laws, from the Official Journal of the European Union
'''
Ignore the square brackets and replace the text inside with the relevant information. For example, if the metadata of the used document is "Source: './data/Finance_laws/EurLex_v1/AMLD_EURLEX.pdf', Page: 0", the source would be 'AMLD_EURLEX.pdf' and the page number would be '0'. 

## CONVERSATION:
User: {user_question}
Agent:

"""

The `get_message()` function creates a **message object** to send to the chatbot API.

1. **Arguments**:
    - `user_question`: The **query** from the user.
    - `k`: The number of **documents to retrieve** for context (default: 10).
    - `openai`: Boolean flag indicating whether to use **OpenAI** or **SentenceTransformer** embeddings.

2. **Logic**:
    - This function calls `get_prompt()` to generate a **full prompt** (question, context, etc.).
    - It then formats the prompt into a **list** with a single message:
      - `role`: Defines the message as coming from the **user**.
      - `content`: Contains the **formatted prompt**.

3. **Output**:
    - A list containing the **message object** that can be passed to the chatbot API for processing.

This ensures that the chatbot receives the user’s question in a structured and clear format, along with the required legal context.


In [10]:
def get_message(user_question, k=10, openai=False):
    """
    Get the initial message for the user question
    
    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings
    
    Returns:
    list: A list containing the initial message
    """
    
    return [{'role': 'user', 'content': get_prompt(user_question, k, openai)}]

## **Generating Responses with RAG**
Now that we have:

✔ **Loaded and embedded the documents**  
✔ **Implemented vector similarity search**  
✔ **Formatted retrieved chunks into a structured prompt**  

We can now use **GPT-4o-mini** to **generate answers** based on the retrieved legal documents.


In [11]:
def get_response(user_question, just_message = True, model = "gpt-4o-mini", k=10, openai = False, **kwargs):
    """
    Get the response to the user question
    
    Args:
    user_question (str): The user question
    just_message (bool): Whether to return the message only or the whole object
    model (str): The model to use for the response
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings
    **kwargs: Additional keyword arguments for the completion, such as `temperature`, `max_tokens`, etc.
    
    Returns:
    str: The response to the user question
    """
    
    message = get_message(user_question, k, openai)

    completion = client.chat.completions.create(
        model=model,
        messages=message,
        **kwargs
    )
    
    if just_message:
        return completion.choices[0].message.content
    
    return completion.choices[0]


---

## **Evaluation**
To test the chatbot's capabilities, a **legal question** was derived from a specific section of the document.


After each question, we evaluate the answers and compare them against the section from which the question was made. We also use **ChatGPT** to give us some metrics for the results. These are the metrics it gave us:

> There are several metrics we can use to evaluate the responses provided by Hugging Face and OpenAI models. Although this type of evaluation involves some level of subjectivity, you can still quantify certain aspects for more structured comparison. Here are some metrics you could consider:
>
> 1. **Correctness** (0-1 scale)
>   - Definition: Does the response accurately reflect the content of the source document? Does it provide factual information consistent with the text in the document?
>   - Evaluation: Assign a score based on how well the response adheres to the original information. A perfect match (fully correct) would get a score of 1, while an inaccurate or irrelevant response would get a score of 0.
> 2. **Completeness** (0-1 scale)
>   - Definition: Does the response fully answer the question, covering all the necessary aspects?
>   - Evaluation: Evaluate how completely the response addresses the question. If it covers everything expected (all key points from the source), give a 1. Partial answers or missing important points could score lower.
> 3. **Conciseness** (0-1 scale)
>   - Definition: Is the response concise and to the point, without unnecessary information or elaboration?
>   - Evaluation: You can score the response higher if it avoids excessive or irrelevant information, and lower if it adds unnecessary detail that could confuse the user.
> 4. **Relevance** (0-1 scale)
>   - Definition: Are the details mentioned in the response directly related to the source paragraph and the question?
>   - Evaluation: Responses that stay on-topic and don't introduce irrelevant points (such as unrelated legal provisions) will score higher. Adding unrelated details lowers the score.
> 5. **Citation Accuracy** (0-1 scale)
>   - Definition: Does the response cite the correct source page or location?
>   - Evaluation: Responses that cite the correct page/location get a full score of 1. If incorrect or off by a significant margin, the score would be reduced.
> 6. **Language Quality** (0-1 scale)
>   - Definition: Is the response grammatically sound, clear, and easy to understand? Does it use professional language appropriate to the context?
>   - Evaluation: Responses with good grammar, clear language, and appropriate terminology score higher. Ambiguous or poorly phrased answers score lower.

Then, it calculates a **weighted scale** with the scores obtained. The weights assigned to each are the following:

<center>

|Metric             |Weight|
|------------------:|------|
| Correctness       |40%   |
| Completeness      |30%   |
| Conciseness       |10%   |
| Relevance         |10%   |
| Citation Accuracy |5%    |
| Language Quality  |5%    |

</center>


### **First Question**
- **Source**: `data/Finance_laws/EurLex_v1/BRRD_EURLEX.pdf`
- **Location**: Page 40, Section 3, Article 10, Paragraph 8

The text outlines the **responsibilities of Member States** in ensuring that resolution authorities have the power to request financial records from institutions. The specific clause mentions the ability of the resolution authority to set time limits for producing these records, with the possibility of different limits for various contract types.

#### Example Text:
> Member States shall ensure that resolution authorities have the power to require an institution and an entity referred to in point (b), (c) or (d) of Article 1(1) to maintain detailed records of financial contracts to which it is a party...

From this **paragraph**, we can form a question to ask the chatbot, such as:
- *"Do authorities have the power to require an institution to mantain records of financial contracts?"*

This demonstrates how **specific sections** of the document are extracted and **transformed into relevant legal questions** for the chatbot to process.


#### Hugging Face Embeddings Response

We use **Hugging Face embeddings** to answer legal questions by retrieving the most relevant document chunks from the database. This process helps ensure that responses are accurate and grounded in legal documents.

##### Response Process
1. **Convert the user's legal question** into an embedding using Hugging Face models.
2. **Perform a similarity search** to retrieve the most relevant legal text from the database.
3. **Generate a response** using the retrieved text and structure it appropriately.


In [ ]:
question = "Do authorities have the power to require an institution to mantain records of financial contracts?"

response = get_response(question)

display(Markdown(response))

# Authority to Require Record-Keeping
Yes, authorities do have the power to require an institution to maintain detailed records of financial contracts to which it is a party. Member States are mandated to ensure that resolution authorities possess this power and may specify time-limits for institutions to be capable of producing those records. The resolution authority can also set different time-limits for different types of financial contracts (BRRD, Article 45(1), page 40).

Source:
- From `BRRD_EURLEX.pdf`, page 40 of the European Union Finance Laws, from the Official Journal of the European Union.

#### OpenAI Embeddings Response

We use **OpenAI embeddings** to handle legal questions by performing a similarity search to find the most relevant chunks of legal text from the database. This method leverages powerful models like **`text-embedding-3-large`** to retrieve and structure answers.

##### Response Process
1. **Convert the user's legal question** into an embedding using OpenAI models.
2. **Perform a similarity search** to retrieve the most relevant legal text.
3. **Generate a response** based on the retrieved text and present it clearly.


In [ ]:
question = "Do authorities have the power to require an institution to mantain records of financial contracts?"

response = get_response(question, openai=True)

display(Markdown(response))

# Authority to Require Maintenance of Financial Records
Yes, competent authorities or resolution authorities have the power to require an institution to maintain detailed records of financial contracts (BRRD_EURLEX.pdf, page 110). This requirement is intended to ensure that the authorities can fulfill their responsibilities effectively.

Source:
- From `BRRD_EURLEX.pdf`, page 110 of the European Union Finance Laws, from the Official Journal of the European Union.

#### **Self-Evaluation of Hugging Face and OpenAI Responses for Q1**



##### **Question Asked**:
*"Do authorities have the power to require an institution to maintain records of financial contracts?"*

##### **Expected Response**:
The expected response was to confirm that authorities (specifically resolution authorities) have the power to require institutions to maintain detailed records of financial contracts. The response should also mention the possibility of specifying time limits for producing those records and the allowance for different time limits for various types of contracts, as outlined in the given paragraph.

##### **Actual Responses**:

**1. Hugging Face Embeddings Response**:
- **Output**:
    > Yes, authorities do have the power to require an institution to maintain detailed records of financial contracts to which it is a party. Member States are mandated to ensure that resolution authorities possess this power and may specify time-limits for institutions to be capable of producing those records. The resolution authority can also set different time-limits for different types of financial contracts (BRRD, Article 45(1), page 40).

- **Evaluation**:
    - **Correctness**: This response correctly answers the question, providing information about the authorities' power and even mentions time limits for records, which is directly relevant to the paragraph.
    - **Completeness**: It covers the main points of the paragraph, including the authorities’ power and the variation of time limits for different financial contracts. However, it doesn't specifically mention the entities referred to in points (b), (c), and (d) of Article 1(1).
    - **Citation**: The response correctly references **page 40**, which aligns with the section from the document.

**2. OpenAI Embeddings Response**:
- **Output**:
    > Yes, competent authorities or resolution authorities have the power to require an institution to maintain detailed records of financial contracts (BRRD_EURLEX.pdf, page 110). This requirement is intended to ensure that the authorities can fulfill their responsibilities effectively.

- **Evaluation**:
    - **Correctness**: This response also correctly answers the question, mentioning the authorities' power to require record maintenance. However, it lacks specific details on the ability to set time limits for records, which was an important aspect of the paragraph.
    - **Completeness**: The response is more general, lacking details about time limits and variations for different contract types. It is a valid response but omits key points expected in a more thorough answer.
    - **Citation**: The response cites **page 110**, which differs from the expected page (40).

---

##### **Strengths**:
- Both **Hugging Face** and **OpenAI** embeddings responses correctly identified the core legal authority to require institutions to maintain financial records.
- The **Hugging Face response** included more relevant details from the source paragraph, including the mention of time limits and different contract types, making it more complete.
- The **OpenAI response** provided a more general answer, which, while correct, missed the finer details provided in the legal text.

##### **Weaknesses**:
- **OpenAI response** cited a different page number (110) than expected. While the general idea was conveyed, it lacked precision.
- Neither response explicitly mentioned the entities outlined in **points (b), (c), or (d) of Article 1(1)**, which were referenced in the paragraph, though this might be acceptable given the broader question scope.

---

#### **Document Results Summary**:

##### **Hugging Face Embeddings**:
- **Correctness**: High — It accurately reflects the legal obligations and powers granted to authorities.
- **Completeness**: High — It covers the essential points, including time limits and the scope of authority, as outlined in the legal paragraph.
- **Citation**: Accurate citation of **page 40**, directly reflecting the document's structure.

##### **OpenAI Embeddings**:
- **Correctness**: Moderate — It correctly identifies the authorities' power but lacks detailed specifics.
- **Completeness**: Moderate — The response is general and misses details on time limits and the different treatment of financial contracts.
- **Citation**: Incorrect — The reference to **page 110** is likely from a different section.

---

#### **Conclusion**:
The **Hugging Face embeddings response** performed better in this scenario due to its more detailed and accurate reflection of the paragraph. The **OpenAI embeddings response**, while correct, was more general and less precise in both content and citation.


#### **ChatGPT Evaluation for Question 1**

| Metric              | Weight | Hugging Face Score | OpenAI Score |
|---------------------|--------|--------------------|--------------|
| **Correctness**      | 0.4    | 0.9                | 0.8          |
| **Completeness**     | 0.3    | 0.95               | 0.85         |
| **Conciseness**      | 0.1    | 0.9                | 0.7          |
| **Relevance**        | 0.1    | 0.9                | 0.75         |
| **Citation Accuracy**| 0.05   | 0.2                | 0.1          |
| **Language Quality** | 0.05   | 0.95               | 0.9          |

---

##### **Weighted Scores Calculation**

- **Hugging Face Weighted Score**: 
$$ 
  = (0.4 * 0.9) + (0.3 * 0.95) + (0.1 * 0.9) + (0.1 * 0.9) + (0.05 * 0.2) + (0.05 * 0.95) \\
  = 0.88 \
$$

- **OpenAI Weighted Score**:  
$$
   = (0.4 * 0.8) + (0.3 * 0.85) + (0.1 * 0.7) + (0.1 * 0.75) + (0.05 * 0.1) + (0.05 * 0.9) \\
   = 0.78 \
$$

In [ ]:
##########################################################################################################################################

### **Second Question**
- **Source** `data/Finance_laws/EurLex_v1/CRR_EURLEX.pdf`
- **Location**: Page 73

We took the following paragraph from the document

> Article 108 
>
> **Use of credit risk mitigation technique under the 
Standardised Approach and the IRB Approach**
> 1. For an exposure to which an institution applies the Standardised Approach under Chapter 2 or applies the IRB Approach under Chapter 3 but without using its own estimates of loss given default (LGD) and conversion factors under Article 151, the institution may use credit risk mitigation in accordance with Chapter 4 in the calculation of risk-weighted exposure amounts for the purposes of points (a) and (f) of Article 92(3) or, as relevant, expected loss amounts for the purposes of the calculation referred to in point (d) of Article 36(1) and point (c) of Article 62. 
> 2. For an exposure to which an institution applies the IRB Approach by using their own estimates of LGD and conversion factors under Article 151, the institution may use credit risk mitigation in accordance with Chapter 3. 

To make the question: ***What is the use of credit risk mitigation technique under the Standardised Approach and the IRB Approach?***

In [ ]:
user_question = "What is the use of credit risk mitigation technique under the Standardised Approach and the IRB Approach?"

hugging_response = get_response(user_question)
openai_response = get_response(user_question, openai=True)

display(Markdown("# Hugging Face Response"))
display(Markdown(hugging_response))

display(Markdown("# OpenAI Response"))
display(Markdown(openai_response))

#### **Self-Evaluation of Hugging Face and OpenAI Responses for Q2**


##### **Question Asked**:
*"What is the use of credit risk mitigation technique under the Standardised Approach and the IRB Approach?"*

##### **Expected Response**:
The expected response is an explanation of how institutions can use credit risk mitigation techniques under both the Standardised Approach and the IRB Approach. For the Standardised Approach, the focus is on how institutions can apply credit risk mitigation to calculate risk-weighted exposure amounts and expected loss amounts. Under the IRB Approach, the institutions use their own estimates of Loss Given Default (LGD) and conversion factors, also applying credit risk mitigation in accordance with the rules in Chapter 3.

##### **Actual Responses**:

**1. Hugging Face Embeddings Response**:
- **Output**:
    > Credit risk mitigation techniques are utilized differently under the Standardised Approach and the Internal Ratings-Based (IRB) Approach.
    > 
    > **Standardised Approach**: Institutions applying the Standardised Approach can use credit risk mitigation techniques to calculate risk-weighted exposure amounts. Specifically, when an exposure is treated under this approach, the institution may incorporate credit risk mitigation in accordance with the relevant provisions, impacting both risk-weighted exposure amounts and expected loss amounts (Article 108(1)).
    > 
    > **IRB Approach**: For institutions using the IRB Approach, credit risk mitigation can also be applied. If an institution utilizes its own estimates of Loss Given Default (LGD) and conversion factors as permitted under Article 151, it can recognize credit risk mitigation strategies in accordance with the respective provisions set out in Chapter 3 (Article 108(2)).
    > 
    > Both approaches aim to effectively manage and reduce the capital requirements related to credit risk by allowing institutions to account for the protections provided by credit risk mitigation techniques (Article 144, CRR).
    > 
    > Source: 
    > 
    > - From CRR_EURLEX.pdf, page 72 of the European Union Finance Laws, from the Official Journal of the European Union
    > - From CRR_EURLEX.pdf, page 93 of the European Union Finance Laws, from the Official Journal of the European Union

- **Evaluation**:
    - **Correctness**: The response correctly answers the question by providing a thorough explanation of how credit risk mitigation is used under both approaches, with reference to the specific articles of the document.
    - **Completeness**: It covers the essential details for both the Standardised Approach and IRB Approach, but it adds some details (like Article 144) not mentioned in the source paragraph.
    - **Citation**: The response cites **pages 72** and **93**. One of them directly correspond to the location of the source paragraph.

**2. OpenAI Embeddings Response**:
- **Output**:
    > Credit risk mitigation techniques can be utilized under both the Standardised Approach and the Internal Ratings-Based (IRB) Approach to adjust the calculations of risk-weighted exposure amounts and expected loss amounts.
    > 
    > **Under the Standardised Approach**: Institutions may use credit risk mitigation when calculating risk-weighted exposure amounts and expected loss amounts, as provided in Article 108(1). They must ensure that they comply with specified provisions, including the treatment of different forms of collateral when covering a single exposure (Article 108(5)).
    > 
    > **Under the IRB Approach**: Institutions that use the IRB Approach, either employing their own estimates of loss given default (LGD) and conversion factors or not, may also apply credit risk mitigation techniques to their calculations, in accordance with the relevant provisions of Chapter 3 (Article 108(2)).
    > 
    > The key condition is that any credit exposure for which credit risk mitigation is applied should have a risk-weighted exposure amount that does not exceed that of an otherwise identical exposure without credit risk mitigation (Article 193(1)). Institutions must also undertake a full credit risk assessment of the underlying exposure, despite having credit risk mitigation in place (Article 108(9)).
    > 
    > Source: 
    > 
    > - From CRR_EURLEX.pdf, page 72 of the European Union Finance Laws, from the Official Journal of the European Union
    > - From CRR_EURLEX.pdf, page 122 of the European Union Finance Laws, from the Official Journal of the European Union

- **Evaluation**:
    - **Correctness**: This response correctly explains how credit risk mitigation is applied under both the Standardised Approach and the IRB Approach. However, it introduces additional concepts (e.g., collateral and full credit risk assessment) that aren't part of the original paragraph.
    - **Completeness**: While the main ideas are covered, the OpenAI response adds unnecessary details, such as Article 193(1) and Article 108(9), which could confuse the focus of the answer.
    - **Citation**: The response cites **pages 72** and **122**. One of them directly correspond to the location of the source paragraph.

---

##### **Strengths**:
- Both responses correctly explain the application of credit risk mitigation techniques under both the **Standardised Approach** and the **IRB Approach**.
- **Hugging Face's response** closely mirrors the information expected from the source paragraph.
- **OpenAI’s response** includes more detailed provisions, though some are not directly related to the source paragraph.

##### **Weaknesses**:
- **OpenAI’s response** included additional details that were not relevant to the specific paragraph.
- Neither response stuck entirely to the expected content of the source paragraph, and both cited information from beyond the immediate text.

---

#### **Document Results Summary**:

##### **Hugging Face Embeddings**:
- **Correctness**: High — It accurately reflects how credit risk mitigation techniques are applied under both approaches.
- **Completeness**: High — It covers the essential points expected in the paragraph but adds some external references like Article 144.
- **Citation**: Incorrect — Cited **pages 72** and **93**.

##### **OpenAI Embeddings**:
- **Correctness**: Moderate — It provides an accurate but overly broad explanation that introduces additional details not required.
- **Completeness**: Moderate — It covers the main points but adds information that was not necessary (e.g., collateral and credit risk assessments).
- **Citation**: Incorrect — Cited **pages 72** and **122**.

---

#### **Conclusion**:
Both **Hugging Face** and **OpenAI embeddings** responses provided reasonable explanations of credit risk mitigation techniques under the Standardised Approach and the IRB Approach. However, **Hugging Face** performed better by staying closer to the source paragraph’s content, while **OpenAI** added unnecessary details and cited a wider range of articles. The page citations were incorrect for both responses.


Here’s the evaluation, including the updated table and weighted scores, based on your instructions regarding the page shift:

---

#### **ChatGPT Evaluation for Question 2**

| **Metric**            | **Hugging Face Score** | **OpenAI Score** |
|-----------------------|------------------------|------------------|
| **Correctness**        | 0.9                    | 0.9              |
| **Completeness**       | 0.85                   | 0.95             |
| **Conciseness**        | 0.9                    | 0.75             |
| **Relevance**          | 0.9                    | 0.85             |
| **Citation Accuracy**  | 0.95                   | 0.95             |
| **Language Quality**   | 0.95                   | 0.9              |

---

##### **Weighted Scores**:

The weights (assuming 40% for Correctness, 30% for Completeness, 10% for Conciseness, 10% for Relevance, 5% for Citation Accuracy, and 5% for Language Quality):

**Hugging Face Weighted Score**:
$$
= (0.4 \times 0.9) + (0.3 \times 0.85) + (0.1 \times 0.9) + (0.1 \times 0.9) + (0.05 \times 0.95) + (0.05 \times 0.95) \\
% = 0.36 + 0.255 + 0.09 + 0.09 + 0.0475 + 0.0475 \\
= 0.8875
$$

**OpenAI Weighted Score**:
$$

= (0.4 \times 0.9) + (0.3 \times 0.95) + (0.1 \times 0.75) + (0.1 \times 0.85) + (0.05 \times 0.95) + (0.05 \times 0.9) \\
% = 0.36 + 0.285 + 0.075 + 0.085 + 0.0475 + 0.045 \\
= 0.8975 \\

$$
---

So, the final weighted scores are:

- **Hugging Face Weighted Score**: 0.8875
- **OpenAI Weighted Score**: 0.8975



In [ ]:
################################################################

### **Third Question**

- **Source** `data/Finance_laws/EurLex_v1/IFR_EURLEX.pdf`
- **Location**: Page 25

We took the following paragraph from the document

> RtC K‐factor requirement 
> The RtC K‐factor requirement is determined by the following formula: 
>
> K‐AUM + K‐CMH + K‐ASA + K‐COH 
>
> where: 
>
> K‐AUM is equal to AUM measured in accordance with Article 17, multiplied by the corresponding coefficient in Article 15 (2); 
>
> K‐CMH is equal to CMH measured in accordance with Article 18, multiplied by the corresponding coefficient in Article 15 (2); 
>
> K‐ASA is equal to ASA measured in accordance with Article 19, multiplied by the corresponding coefficient in Article 15 (2); 
>
> K‐COH is equal to COH measured in accordance with Article 20, multiplied by the corresponding coefficient in Article 15 (2).

To make the question: ***What are RtC K-factor requirements?***

In [13]:
question = "What are RtC K-factor requirements?"

hugging_response = get_response(question)
openai_response = get_response(question, openai=True)

display(Markdown("# Hugging Face Response"))
display(Markdown(hugging_response))

display(Markdown("# OpenAI Response"))
display(Markdown(openai_response))

# Hugging Face Response

# RtC K-factor Requirements
The RtC K-factor requirement is determined by the following formula: 

K = K-A UM + K-CMH + K-ASA + K-COH 

where:
- K-A UM is equal to A UM measured in accordance with Article 17, multiplied by the corresponding coefficient in Article 15(2);
- K-CMH is equal to CMH measured in accordance with Article 18, multiplied by the corresponding coefficient in Article 15(2);
- K-ASA is equal to ASA measured in accordance with Article 19, multiplied by the corresponding coefficient in Article 15(2);
- K-COH is equal to COH measured in accordance with Article 20, multiplied by the corresponding coefficient in Article 15(2) (source).

Source:
- From `IFR_EURLEX.pdf, page 37` of the European Union Finance Laws, from the Official Journal of the European Union.

# OpenAI Response

# RtC K-factor Requirements
The RtC K-factor requirements are defined by a specific formula as follows:

\[ K\text{-RtC} = K\text{-A UM} + K\text{-CMH} + K\text{-ASA} + K\text{-COH} \]

Where:
- **K-A UM** represents the risk associated with client assets under management. It is calculated as the rolling average of the total monthly assets under management for the last 15 months, excluding the three most recent monthly values and converted into the entity’s functional currency.
- **K-CMH** refers to the client money held by the investment firm, measured similarly to K-A UM.
- **K-ASA** signifies assets safeguarded and administered, calculated as per its respective regulation.
- **K-COH** denotes the client orders handled, which is the rolling average of the total daily client orders over the previous six months, excluding the three most recent months.

Investment firms are required to evaluate these K-factors regularly and inform their competent authority of any material changes in their own funds requirement. The K-factors under RtC are important as they encapsulate various risks that investment firms face in relation to client exposures (Article 16, 20, and 21 from the regulation).

Source:
- From `IFR_EURLEX.pdf`, page 25 of the European Union Finance Laws, from the Official Journal of the European Union
- From `IFR_EURLEX.pdf`, page 5 of the European Union Finance Laws, from the Official Journal of the European Union

##### **Question Asked**:
*"What are RtC K-factor requirements?"*

##### **Expected Response**:
The expected response should outline the formula used to determine the RtC K-factor requirements, with an explanation of the components: K-AUM, K-CMH, K-ASA, and K-COH. Each of these components should be described based on the articles mentioned in the document (Article 17, Article 18, etc.), as they correspond to financial measurements.

##### **Actual Responses**:

**1. Hugging Face Embeddings Response**:
- **Output**:
    > The RtC K-factor requirements are determined by the formula: K-AUM + K-CMH + K-ASA + K-COH, where:
    > - K-AUM is equal to AUM (Assets Under Management) measured in accordance with Article 17, multiplied by the corresponding coefficient in Article 15(2).
    > - K-CMH is equal to CMH (Client Money Held) measured in accordance with Article 18, multiplied by the corresponding coefficient in Article 15(2).
    > - K-ASA is equal to ASA (Assets Under Management) measured in accordance with Article 19, multiplied by the corresponding coefficient in Article 15(2).
    > - K-COH is equal to COH (Cash Held) measured in accordance with Article 20, multiplied by the corresponding coefficient in Article 15(2).
    >
    > (source: IFR_EURLEX.pdf, page 36).

- **Evaluation**:
    - **Correctness**: This response correctly outlines the components of the RtC K-factor requirements, providing relevant details from the source text.
    - **Completeness**: It fully explains the main components (K-AUM, K-CMH, K-ASA, and K-COH) and cites the proper articles from the document. The description is clear and concise.
    - **Citation**: The response cites **page 36**, but the location in the source is **page 25**, making this citation inaccurate.

**2. OpenAI Embeddings Response**:
- **Output**:
    > The RtC K-factor requirements for investment firms are determined by the following formula:
    > - **K-RtC = K-AUM + K-CMH + K-ASA + K-COH**, 
    >
    > where:
    > - **K-AUM** refers to the K-factor for assets under management (AUM), calculated based on the rolling average of total monthly assets under management over the last 15 months.
    > - **K-CMH** corresponds to client money held (CMH), calculated based on a specified methodology.
    > - **K-ASA** is for assets safeguarded and administered (ASA), also calculated in a prescribed manner.
    > - **K-COH** relates to client orders handled (COH), which is based on the rolling average of daily client orders over the last six months (excluding the three most recent months) (Article 16).
    >
    > Additionally, investment firms must monitor their K-factors for trends that could lead to materially different own funds requirements and must notify their competent authority if such changes occur (source: Article 24).

- **Evaluation**:
    - **Correctness**: The response is correct in its overall explanation of the K-factor requirements and mentions relevant components. However, it introduces additional details not found in the original source, such as the rolling average and monitoring trends.
    - **Completeness**: While the response provides an informative answer, it includes details beyond the scope of the paragraph and document provided. It does not align with the content from **page 25** and could cause confusion.
    - **Citation**: The response cites multiple pages (**24** and **4**). One of them directly corresponds to the location of the source paragraph.

---

##### **Strengths**:
- Both **Hugging Face** and **OpenAI** embeddings responses provided correct descriptions of the RtC K-factor requirements.
- The **Hugging Face response** followed the source more closely, sticking to the articles mentioned in the document.
- The **OpenAI response** provided an additional context that, while insightful, was not required based on the specific paragraph cited.

##### **Weaknesses**:
- One response cited incorrect pages, with **Hugging Face** mentioning **page 36** and **OpenAI** referring to **page 4**.
- The **OpenAI response** introduced extra information (e.g., rolling averages and monitoring) that went beyond the question's scope.

---

#### **Document Results Summary**:

##### **Hugging Face Embeddings**:
- **Correctness**: High — The response closely follows the source document, with accurate information.
- **Completeness**: High — The essential details of the RtC K-factor requirements are clearly outlined, without introducing unnecessary information.
- **Citation**: Inaccurate — The response incorrectly references **page 36** instead of **page 25**.

##### **OpenAI Embeddings**:
- **Correctness**: Moderate — The response correctly identifies the components of the RtC K-factor but includes extra information not needed.
- **Completeness**: Moderate — While the response is informative, it goes beyond the necessary scope and misses focus on the exact paragraph.
- **Citation**: Inaccurate — The citation refers to **pages 24 and 4**. One of them directly corresponds to the location of the source paragraph.

---

#### **Conclusion**:
The **Hugging Face embeddings response** is more accurate and aligned with the source material. Although both responses provided valid information, the **OpenAI embeddings response** introduced unnecessary additional details and cited incorrect pages.

---

##### **Metrics Evaluation Table**:


| Metric              | Weight | Hugging Face Score | OpenAI Score |
|---------------------|--------|--------------------|--------------|
| **Correctness**      | 0.4    | 0.85               | 0.9          |
| **Completeness**     | 0.3    | 0.9                | 0.85         |
| **Conciseness**      | 0.1    | 0.8                | 0.75         |
| **Relevance**        | 0.1    | 0.9                | 0.85         |
| **Citation Accuracy**| 0.05   | 0.6                | 0.85         |
| **Language Quality** | 0.05   | 0.95               | 0.9          |


#### **Weighted Scores Calculation**

- **Hugging Face Weighted Score**:
$$  
  = (0.4 * 0.85) + (0.3 * 0.9) + (0.1 * 0.8) + (0.1 * 0.9) + (0.05 * 0.6) + (0.05 * 0.95) \\
  = 0.865
$$
- **OpenAI Weighted Score**:
$$  
  = (0.4 * 0.9) + (0.3 * 0.85) + (0.1 * 0.75) + (0.1 * 0.85) + (0.05 * 0.85) + (0.05 * 0.9) \\
  = 0.86
$$


In [ ]:
#########################################

### **Fourth Question**

- **Source** `data/Finance_laws/EurLex_v1/DGSD_EURLEX.pdf`
- **Location**: Page 5

We found a simple line, and made a question from it:

> (35) DGSs should invest in low-risk assets. 

So, we asked: ***In what type of assests shoud DGSs be invested?***

In [12]:
question = "In what type of assests shoud DGSs be invested?"

hugging_response = get_response(question)
openai_response = get_response(question, openai=True)

display(Markdown("Hugging Face Response"))
display(Markdown(hugging_response))

display(Markdown("---"))
display(Markdown("OpenAI Response"))
display(Markdown(openai_response))

Hugging Face Response

# Types of Assets for DGS Investments
Deposit Guarantee Schemes (DGSs) should primarily invest in cash, deposits, payment commitments, and low-risk assets that can be liquidated within a short period of time. It is necessary that these assets are of low risk to ensure adequate financial means are available for fulfilling their obligations (DGSD, Article 10). 

Additionally, DGSs must have adequate alternative funding arrangements in place to obtain short-term funding to meet claims made against them (DGSD, Article 10).

Source:
- From `DGSD_EURLEX.pdf`, page 5 of the European Union Finance Laws, from the Official Journal of the European Union.
- From `DGSD_EURLEX.pdf`, page 19 of the European Union Finance Laws, from the Official Journal of the European Union.

---

OpenAI Response

# Investment Assets for DGSs
Deposit Guarantee Schemes (DGSs) should primarily focus on assets that enhance their financial stability and ability to meet their obligations. The regulations indicate that DGSs are expected to be financially sound and transparent in their operations and governance. While the specific types of assets for investment are not directly enumerated in the provided context, it is implied that DGSs should use available financial means in a manner that ensures they can repay depositors and meet claims effectively.

DGSs are allowed to use their financial means primarily for repaying depositors as mandated under Article 11. Additionally, they may finance the resolution of credit institutions under specific regulatory frameworks. The investment strategy should be aligned with maintaining adequate liquidity and managing risks effectively (source: DGSD_EURLEX.pdf, page 11, 18).

Furthermore, DGSs must ensure that their available financial means are sufficient to cover potential liabilities, as outlined in Article 10. This necessitates investing in assets that are stable and liquid, allowing for quick access when claims arise.

Source:
- From `DGSD_EURLEX.pdf`, page `11` of the European Union Finance Laws, from the Official Journal of the European Union
- From `DGSD_EURLEX.pdf`, page `18` of the European Union Finance Laws, from the Official Journal of the European Union


---

##### **Question Asked**:
*In what type of assets should DGSs be invested?*

##### **Expected Response**:
The expected response was to indicate that Deposit Guarantee Schemes (DGSs) should primarily invest in low-risk assets, as stated in the source document. These assets must ensure the financial stability and liquidity of the scheme to meet its obligations. The response should mention specific types of low-risk assets, like cash, deposits, or payment commitments, and emphasize the necessity for assets to be liquidated quickly.

##### **Actual Responses**:

**1. Hugging Face Embeddings Response**:
- **Output**:
    > Deposit Guarantee Schemes (DGSs) should primarily invest in cash, deposits, payment commitments, and low-risk assets that can be liquidated within a short period of time. It is necessary that these assets are of low risk to ensure adequate financial means are available for fulfilling their obligations (DGSD, Article 10). 
    >
    > Additionally, DGSs must have adequate alternative funding arrangements in place to obtain short-term funding to meet claims made against them (DGSD, Article 10).

- **Evaluation**:
    - **Correctness**: This response is accurate, specifying the required investment in low-risk assets and citing the correct article. It also includes the necessity for alternative funding arrangements.
    - **Completeness**: The response is complete, mentioning the key assets DGSs should invest in, as well as additional considerations like alternative funding.
    - **Citation**: The response references **page 5**, which is appropriate given the source document.

**2. OpenAI Embeddings Response**:
- **Output**:
    > Deposit Guarantee Schemes (DGSs) should primarily focus on assets that enhance their financial stability and ability to meet their obligations. The regulations indicate that DGSs are expected to be financially sound and transparent in their operations and governance. While the specific types of assets for investment are not directly enumerated in the provided context, it is implied that DGSs should use available financial means in a manner that ensures they can repay depositors and meet claims effectively.
    >
    > DGSs are allowed to use their financial means primarily for repaying depositors as mandated under Article 11. Additionally, they may finance the resolution of credit institutions under specific regulatory frameworks. The investment strategy should be aligned with maintaining adequate liquidity and managing risks effectively (source: DGSD_EURLEX.pdf, page 11, 18).
    >
    > Furthermore, DGSs must ensure that their available financial means are sufficient to cover potential liabilities, as outlined in Article 10. This necessitates investing in assets that are stable and liquid, allowing for quick access when claims arise.

- **Evaluation**:
    - **Correctness**: The response is correct in its general assessment of the financial responsibilities of DGSs. However, it doesn't mention specific types of low-risk assets, which is crucial to the question.
    - **Completeness**: The response provides a broader view of DGSs’ obligations but omits the specific types of assets mentioned in the source text. It is more abstract in comparison to the Hugging Face response.
    - **Citation**: The response references **page 11** and **page 18**, which are different from the expected page (5) and may reference other relevant sections, though they do not directly address the specific asset types.

---

##### **Strengths**:
- **Hugging Face response** was more specific, mentioning the exact types of assets (cash, deposits, payment commitments) and the relevant article.
- **OpenAI response** provided a broader context of DGSs' financial responsibilities, including repayment obligations and the need for liquidity.

##### **Weaknesses**:
- **OpenAI response** was more general and did not provide specific types of low-risk assets, which were directly mentioned in the source.
- **OpenAI citation** pointed to different pages than the one where the primary information was expected (page 5).

---

##### **ChatGPT Evaluation for Question 4**

| **Metric**            | **Hugging Face Score** | **OpenAI Score** |
|-----------------------|------------------------|------------------|
| **Correctness**        | 0.95                   | 0.75             |
| **Completeness**       | 0.9                    | 0.7              |
| **Conciseness**        | 0.9                    | 0.75             |
| **Relevance**          | 0.9                    | 0.75             |
| **Citation Accuracy**  | 0.95                   | 0.6              |
| **Language Quality**   | 0.9                    | 0.9              |



##### **Weighted Scores**:

The weights (assuming 40% for Correctness, 30% for Completeness, 10% for Conciseness, 10% for Relevance, 5% for Citation Accuracy, and 5% for Language Quality):

**Hugging Face Weighted Score**:
$$
= (0.4 \times 0.95) + (0.3 \times 0.9) + (0.1 \times 0.9) + (0.1 \times 0.9) + (0.05 \times 0.95) + (0.05 \times 0.9) \\
= 0.38 + 0.27 + 0.09 + 0.09 + 0.0475 + 0.045 \\
= 0.9225
$$

**OpenAI Weighted Score**:
$$
= (0.4 \times 0.75) + (0.3 \times 0.7) + (0.1 \times 0.75) + (0.1 \times 0.75) + (0.05 \times 0.6) + (0.05 \times 0.9) \\
= 0.3 + 0.21 + 0.075 + 0.075 + 0.03 + 0.045 \\
= 0.735
$$

---

#### **Conclusion**:
The **Hugging Face embeddings response** outperformed the OpenAI response in both accuracy and specificity. The Hugging Face response not only included the specific asset types but also provided a more relevant citation. While the OpenAI response was correct in general, it lacked the precise details required to fully answer the question. Based on the weighted scores, **Hugging Face** scores **0.9225** while **OpenAI** scores **0.735**, making Hugging Face the better option in this scenario.